This notebook describes:
1. How to generate training/validation dataset
2. How to build a simple inference network
3. Inference

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
np.random.get_state(42)

from tqdm import tqdm
import pandas as pd
import joblib
import xarray as xr
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression 
from scipy import stats
import xgboost as xgb
from xgboost import plot_importance
import seaborn as sns

import cuml
import xgboost

# Generate Training/Validation sets

In [ ]:
# Process harmonized samples to generate pixel-level data points
x_bands = ['band_01', 'band_02', 'band_03', 'band_04', 'band_05', 'band_06', 'band_07', 'band_08', 'band_8A', 'band_09', 'band_11', 'band_12', 'band_VV', 'band_VH', 'sif_771']
datasets = {'rh': ['/data/global_harmonized', './data/validation/harmonized'],
            'agb': ['/data/global_harmonized', './data/validation_agb/harmonized']
           }

names = ['training', 'validation']

for target in datasets.keys():
    dataset_dirs = datasets[target]
    for n, dataset in enumerate(dataset_dirs):
        all_data = []
        harmonized_files = [os.path.join(dataset, file) for file in os.listdir(dataset) if file.endswith('.nc')]

        for i in tqdm(range(len(harmonized_files))):
            try:
                curr_data = []
                ds = xr.open_dataset(harmonized_files[i])
                mask = ds[target].notnull()
                for varname in x_bands:
                    curr_data.append(ds[varname].values[mask])

                curr_data.append(ds[target].values[mask])

                try:
                    all_data = np.concatenate((all_data, curr_data), axis=1)
                except:
                    all_data = curr_data
            except:
                print(f'Fails: {harmonized_files[i]}')

        all_data = np.array(all_data).swapaxes(0,1)
        out_path = f'./data/ml_data/{names[n]}_{target}_global.npy'
        np.save(out_path, all_data)

  6%|▌         | 5789/99913 [15:22<5:57:20,  4.39it/s] 

# Fitting and Inference

In [3]:
def get_pixel_data(train_path, val_path, test_size=0.1, val_ratio=0.9, percentile=95, target='agb'):
    # Split training-testing set (90-10 split); 
    train_data = np.load(train_path)
    
    # Randomly sample validation samples since we have significantly more val than train
    val_data = np.load(val_path)
    val_idx = np.random.choice(np.arange(0, len(val_data)), int(len(train_data)*0.1)) 
    val_data = val_data[val_idx] 
    
    # Filtering to remove outlier (95th percentile and negative or zero values for AGB & >3 meter for RH)
    if target == 'rh':
        # Train on RH > 3 meters
        train_data = train_data[train_data[:,-1] > 3]
        val_data = val_data[val_data[:,-1] > 3]
        y_train,y_test,y_val = np.clip(y_train,0,200),np.clip(y_test,0,200),np.clip(y_val,0,200)
    else:
        # Remove AGB <= 0 & outlier: 95th percentile
        train_data = train_data[train_data[:,-1] > 0]
        train_outlier = np.percentile(train_data[:,-1], percentile)
        train_data = train_data[train_data[:,-1] < train_outlier]
        
        val_data = val_data[val_data[:,-1] > 0]
        val_outlier = np.percentile(val_data[:,-1], percentile)
        val_data = val_data[val_data[:,-1] < val_outlier]
    
    train_data, test_data = train_test_split(train_data, test_size=test_size)
    X_train, y_train = train_data[:,:-1], train_data[:,-1]
    X_test, y_test = test_data[:,:-1], test_data[:,-1]
    X_val, y_val = val_data[:,:-1], val_data[:,-1]
    
    # Ensure no NaN values in the X features since our ML models can't handle such value (assign 0, otherwise)
    return np.nan_to_num(X_train), y_train, np.nan_to_num(X_test), y_test, np.nan_to_num(X_val), y_val

In [4]:
def evaluate_model(model, x, y, target):
    """Evaluate models in terms of rmse, mae, r2"""
    allometric = lambda x: 0.8245 * np.power(np.clip(x,0,200), 1.573) # DEPRECATED
    if target == 'rh':
        y = np.clip(y,0,200)
    
    preds = model.predict(x)
    r2 = model.score(x, y)
    
    if target == 'rh':
        y = allometric(y)
        preds = allometric(preds)
        
    rmse = np.sqrt(mean_squared_error(y, preds))
    mae = mean_absolute_error(y, preds)
    
    return rmse, mae, r2

In [13]:
def evaluate_all(model, train_path, val_path, inputs, target, runs=5, save=None):
    """Encompassing function that runs models across runs (default=5)"""

    assert inputs in ['radar', 'optic', 'all'] # radar: S1/S2, optic: S2-only, all: SIF/S1/S2
    assert save in [None, 'xgb', 'rf']
    
    rmse_test, rmse_val = [], []
    mae_test, mae_val = [], []

    for i in range(runs):
        X_train, y_train, X_test, y_test, X_val, y_val = get_pixel_data(train_path, val_path, target=target)
        if inputs == 'radar':
            X_train, X_test, X_val = X_train[:,:-1], X_test[:,:-1], X_val[:,:-1]
        elif inputs == 'optic':
            X_train, X_test, X_val = X_train[:,:-3], X_test[:,:-3], X_val[:,:-3]
        
        model.fit(X_train.astype(np.float32), y_train.astype(np.float32))

        # Testing (on set-aside GEDI)
        rmse, mae, r2_test = evaluate_model(model, X_test, y_test, target=target)
        rmse_test.append(rmse); mae_test.append(mae)

        # Testing (on ALS)
        rmse, mae, r2_val = evaluate_model(model, X_val, y_val, target=target)
        rmse_val.append(rmse); mae_val.append(mae)
        
        if save != None:
            if save == 'xgb':
                model.save_model(f'./zoo/global/xgboost_{target}_{i+1}.json')
            else:
                joblib.dump(model, f'./zoo/global/rf_{target}_{i+1}.joblib')

    print(u'RMSE for set aside GEDI: {:.2f} \u00B1 {:.2f} \t MAE: {:.2f} \u00B1 {:.2f} \t R2: {:.2f}'.format(np.array(rmse_test).mean(), np.array(rmse_test).std(), np.array(mae_test).mean(), np.array(mae_test).std(), r2_test))
    print(u'RMSE for ALS sites: {:.2f} \u00B1 {:.2f} \t MAE: {:.2f} \u00B1 {:.2f} \t R2: {:.2f}'.format(np.array(rmse_val).mean(), np.array(rmse_val).std(), np.array(mae_val).mean(), np.array(mae_val).std(), r2_val))
        
    return model

In [6]:
def hyp_opt(model, parameters, x, y):
    grid_search = RandomizedSearchCV(
        estimator = model,
        param_distributions = parameters,
        scoring = 'neg_mean_squared_error',
        n_iter = 50,
        cv = 5,
        verbose = 2,
        random_state = 42
    )

    grid_search.fit(x, y)
    return grid_search.best_estimator_

# Train and Validate

## Optimization

In [7]:
target = 'agb'
train_path = './data/ml_data/training_agb_global.npy'
val_path = './data/ml_data/validation_agb_global.npy'

In [ ]:
# Optimization: XGB
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist')
X_train, y_train, X_test, y_test, X_val, y_val = get_pixel_data(train_path, val_path, target=target)

parameters = {
    'max_depth': range(2, 11, 1),
    'n_estimators': range(200, 2001, 200),
    'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
}

best_params = hyp_opt(xgb_model, parameters, X_train, y_train)
print(best_params) # learning_rate=0.06, n_estimators=800, max_depth=10

In [ ]:
# Optimization: RF
rf_model = cuml.ensemble.RandomForestRegressor()
X_train, y_train, X_test, y_test, X_val, y_val = get_pixel_data(train_path, val_path, target=target)
parameters = {
    'max_depth': range(2, 11, 1),
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'min_samples_split': [2, 4, 6, 8, 10],
    'n_estimators': range(200, 2001, 200)
}

best_params = hyp_opt(rf_model, parameters, X_train, y_train)
print(best_params)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


/home/juanat/anaconda3/envs/rapids/lib/python3.9/site-packages/cuml/internals/api_decorators.py:567: UserWarning: To use pickling or GPU-based prediction first train using float32 data to fit the estimator
  ret_val = func(*args, **kwargs)
/home/juanat/anaconda3/envs/rapids/lib/python3.9/contextlib.py:79: UserWarning: GPU based predict only accepts np.float32 data. The model was trained on np.float64 data hence cannot use GPU-based prediction! 
Defaulting to CPU-based Prediction. 
To predict on float-64 data, set parameter predict_model = 'CPU'
  return func(*args, **kwds)


[CV] END max_depth=5, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time= 1.4min


## Fitting

In [8]:
target = 'agb'

train_path = f'./data/ml_data/training_{target}_global.npy'
val_path = f'./data/ml_data/validation_{target}_global.npy'

### a. Linear Regression

In [9]:
# SIF/S1/S2
lingress = LinearRegression()
lingress = evaluate_all(lingress, train_path, val_path, inputs='all', target=target, save=None)

RMSE for set aside GEDI: 104.20 ± 0.07 	 MAE: 84.50 ± 0.06 	 R2: 0.12
RMSE for ALS sites: 116.17 ± 0.09 	 MAE: 102.47 ± 0.09 	 R2: 0.03


In [10]:
# S1/S2
lingress = LinearRegression()
lingress = evaluate_all(lingress, train_path, val_path, inputs='radar', target=target, save=None)

RMSE for set aside GEDI: 105.20 ± 0.10 	 MAE: 85.44 ± 0.08 	 R2: 0.11
RMSE for ALS sites: 122.61 ± 0.07 	 MAE: 108.06 ± 0.10 	 R2: -0.08


In [11]:
# S2
lingress = LinearRegression()
lingress = evaluate_all(lingress, train_path, val_path, inputs='optic', target=target, save=None)

RMSE for set aside GEDI: 105.47 ± 0.05 	 MAE: 85.69 ± 0.06 	 R2: 0.10
RMSE for ALS sites: 123.90 ± 0.07 	 MAE: 109.25 ± 0.08 	 R2: -0.10


### b. XGBoost

In [15]:
# SIF/S1/S2
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', learning_rate=0.06, n_estimators=800, max_depth=10)
xgb_model = evaluate_all(xgb_model, train_path, val_path, inputs='all', target=target, save='xgb')

RMSE for set aside GEDI: 98.11 ± 0.09 	 MAE: 77.60 ± 0.08 	 R2: 0.22
RMSE for ALS sites: 109.69 ± 0.17 	 MAE: 87.19 ± 0.16 	 R2: 0.14


In [16]:
# S1/S2
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', learning_rate=0.06, n_estimators=800, max_depth=10)
xgb_model = evaluate_all(xgb_model, train_path, val_path, inputs='radar', target=target, save=None)

RMSE for set aside GEDI: 101.29 ± 0.03 	 MAE: 80.63 ± 0.04 	 R2: 0.18
RMSE for ALS sites: 124.66 ± 0.08 	 MAE: 109.69 ± 0.10 	 R2: -0.12


In [17]:
# S2
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist', learning_rate=0.06, n_estimators=800, max_depth=10)
xgb_model = evaluate_all(xgb_model, train_path, val_path, inputs='optic', target=target, save=None)

RMSE for set aside GEDI: 101.88 ± 0.06 	 MAE: 81.13 ± 0.02 	 R2: 0.16
RMSE for ALS sites: 125.06 ± 0.12 	 MAE: 110.23 ± 0.12 	 R2: -0.12


### c. RF

In [9]:
# SIF/S1/S2
rf_model = cuml.ensemble.RandomForestRegressor()
rf_model = evaluate_all(rf_model, train_path, val_path, inputs='all', target=target, save='rf')

RMSE for set aside GEDI: 98.67 ± 0.07 	 MAE: 78.27 ± 0.06 	 R2: 0.21
RMSE for ALS sites: 110.00 ± 0.23 	 MAE: 87.34 ± 0.24 	 R2: 0.13


In [10]:
# S1/S2
rf_model = cuml.ensemble.RandomForestRegressor()
rf_model = evaluate_all(rf_model, train_path, val_path, inputs='radar', target=target, save=None)

RMSE for set aside GEDI: 101.92 ± 0.08 	 MAE: 81.42 ± 0.07 	 R2: 0.16
RMSE for ALS sites: 124.77 ± 0.10 	 MAE: 109.99 ± 0.09 	 R2: -0.12


In [11]:
# S2
rf_model = cuml.ensemble.RandomForestRegressor()
rf_model = evaluate_all(rf_model, train_path, val_path, inputs='optic', target=target, save=None)

RMSE for set aside GEDI: 102.47 ± 0.05 	 MAE: 81.82 ± 0.09 	 R2: 0.15
RMSE for ALS sites: 125.02 ± 0.09 	 MAE: 110.18 ± 0.10 	 R2: -0.12


# Results

For CONUS

1. RMSE:
<table>
    <thead>
        <tr>
            <th>Model</th>
            <th>Inputs</th>
            <th>Test</th>
            <th>Validation</th>
            <th>Autumn</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan=3>Linear regression</td>
            <td>SIF/S1/S2</td>
            <td>66.07 ± 0.06</td>
            <td>81.95 ± 0.01</td>
            <td>81.53 ± 0.00</td>
        </tr>
        <tr>
            <td>S1/S2</td>
            <td>66.46 ± 0.10</td>
            <td>84.33 ± 0.00</td>
            <td>83.72 ± 0.00</td>
        </tr>
        <tr>
            <td>S2-only</td>
            <td>67.10 ± 0.11</td>
            <td>90.99 ± 0.03</td>
            <td>87.20 ± 0.00</td>
        </tr>
        <tr>
            <td rowspan=3>XGBoost</td>
            <td>SIF/S1/S2</td>
            <td>56.66 ± 0.06</td>
            <td>53.37 ± 0.05</td>
            <td>75.88 ± 0.22</td>
        </tr>
        <tr>
            <td>S1/S2</td>
            <td>57.35 ± 0.05</td>
            <td>54.74 ± 0.03</td>
            <td>72.10 ± 0.07</td>
        </tr>
        <tr>
            <td>S2-only</td>
            <td>57.82 ± 0.02</td>
            <td>54.81 ± 0.26</td>
            <td>73.13 ± 0.24</td>
        </tr>
        <tr>
            <td rowspan=3>Random Forest</td>
            <td>SIF/S1/S2</td>
            <td>57.16 ± 0.05</td>
            <td>52.30 ± 0.03</td>
            <td>81.66 ± 0.08</td>
        </tr>
        <tr>
            <td>S1/S2</td>
            <td>58.05 ± 0.03</td>
            <td>54.72 ± 0.06</td>
            <td>75.44 ± 0.08</td>
        </tr>
        <tr>
            <td>S2-only</td>
            <td>58.12 ± 0.02</td>
            <td>54.88 ± 0.18</td>
            <td>76.90 ± 0.01</td>
        </tr>
        <tr>
            <td rowspan=3>UNet</td>
            <td>SIF/S1/S2</td>
            <td><b>48.83 ± 0.19</b></td>
            <td><b>37.93 ± 1.36</b></td>
            <td>66.94 ± 0.51</td>
        </tr>
        <tr>
            <td>S1/S2</td>
            <td>49.30 ± 0.18</td>
            <td>41.99 ± 3.23</td>
            <td><b>65.88 ± 0.27</b></td>
        </tr>
        <tr>
            <td>S2-only</td>
            <td>50.35 ± 0.43</td>
            <td>45.93 ± 2.25</td>
            <td>73.05 ± 1.86</td>
        </tr>
    </tbody>
</table>